In [16]:
# import basic modules
import sys
import os
import time
import numpy as np
from tqdm import tqdm
import gc
import torch
import argparse
import skimage.transform

# import custom modules
root_dir   = '/user_data/mmhender/imStat/'
sys.path.append(os.path.join(root_dir,'code'))
from model_src import fwrf_fit as fwrf_fit
from model_src import fwrf_predict as fwrf_predict
from model_src import texture_statistics_gabor, texture_statistics_pyramid, bdcn_features
from utils import nsd_utils, roi_utils

bdcn_path = '/user_data/mmhender/toolboxes/BDCN/'

from model_fitting import initialize_fitting2 as initialize_fitting
from model_fitting import merge_features, arg_parser

fpX = np.float32
device = 'cpu:0'
# device = initialize_fitting.init_cuda()

# fitting_type='gabor_solo'
fitting_type = 'texture_gabor'
# fitting_type='pyramid'
subject=1
volume_space = False
up_to_sess = 1
n_ori = 4
n_sf = 4
nonlin_fn = False
padding_mode = 'circular'
group_all_hl_feats = False; \
sample_batch_size = 50; voxel_batch_size = 100; \
zscore_features = True; ridge = True; \
shuffle_images = False; random_images = False; random_voxel_data = False; \
do_fitting = True; do_val = True; do_varpart = True; date_str = None;
shuff_rnd_seed = 0; debug = True; \
do_pca = False; min_pct_var = 99; max_pc_to_retain = 400; map_ind = -1; \
n_prf_sd_out = 2; mult_patch_by_prf = True; \
downsample_factor = 1.0; do_nms = False

In [17]:
if 'pyramid' in fitting_type:
    model_name, feature_types_exclude = initialize_fitting.get_pyramid_model_name(ridge, n_ori, n_sf)

    if 'plus_bdcn' in fitting_type:
        model_name2 = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)
        model_name = model_name + '_plus_' + model_name2

elif 'bdcn' in fitting_type:
    model_name = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)

elif 'texture_gabor' in fitting_type:

    model_name = initialize_fitting.get_gabor_texture_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = []

elif 'gabor_solo' in fitting_type:

    model_name = initialize_fitting.get_gabor_solo_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = ['pixel', 'simple_feature_means', 'autocorrs', 'crosscorrs']
else:     
    raise ValueError('your string for fitting_type was not recognized')

In [3]:
# decide what voxels to use  
voxel_mask, voxel_index, voxel_roi, voxel_ncsnr, brain_nii_shape = roi_utils.get_voxel_roi_info(subject, volume_space)

sessions = np.arange(0,up_to_sess)
zscore_betas_within_sess = True
# get all data and corresponding images, in two splits. always fixed set that gets left out
trn_stim_data, trn_voxel_data, val_stim_data, val_voxel_data, image_order = nsd_utils.get_data_splits(subject, sessions=sessions, \
                                                                     voxel_mask=voxel_mask, volume_space=volume_space, \
                                                                      zscore_betas_within_sess=zscore_betas_within_sess, \
                                                                      shuffle_images=shuffle_images, random_images=random_images, \
                                                                                         random_voxel_data=random_voxel_data)



Surface space: ROI defs are located at: /lab_data/tarrlab/common/datasets/NSD/nsddata/freesurfer/subj01/label

18511 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25.]
0 voxels of overlap between face and place definitions, using place defs
unique values in categ labels:
[ 0. 26. 27. 28. 30. 31. 32. 33.]
7047 voxels are defined (differently) in both retinotopic areas and category areas

66545 voxels are defined across all areas, and will be used for analysis

Loading numerical label/name mappings for all ROIs:
[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'VO1', 'VO2', 'PHC1', 'PHC2', 'TO2', 'TO1', 'LO2', 'LO1', 'V3B', 'V3A', 'IPS0', 'IPS1', 'IPS2', 'IPS3', 'IPS4',

In [4]:

if 'pyramid' in fitting_type:
    # Need a multiple of 8
    process_at_size=240
    trn_stim_data = skimage.transform.resize(trn_stim_data, output_shape=(trn_stim_data.shape[0],1,process_at_size, process_at_size))
    val_stim_data = skimage.transform.resize(val_stim_data, output_shape=(val_stim_data.shape[0],1,process_at_size, process_at_size))

# More params for fitting
holdout_size, lambdas = initialize_fitting.get_fitting_pars(trn_voxel_data, zscore_features, ridge=ridge)
# Params for the spatial aspect of the model (possible pRFs)
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    



Possible lambda values are:
[1.0000000e+00 4.2169652e+00 1.7782795e+01 7.4989418e+01 3.1622775e+02
 1.3335215e+03 5.6234131e+03 2.3713736e+04 1.0000000e+05]
most extreme RF positions:
[-0.55 -0.55  0.04]
[0.55       0.55       0.40000001]


In [41]:
_gabor_ext_complex.real_filters_tnsr.device

device(type='cpu')

In [18]:
# Set up the Gabor filtering modules
_gabor_ext_complex, _gabor_ext_simple, _fmaps_fn_complex, _fmaps_fn_simple = \
        initialize_fitting.get_gabor_feature_map_fn(n_ori, n_sf, padding_mode=padding_mode, device=device, \
                                                             nonlin_fn=nonlin_fn);    
# Initialize the "texture" model which builds on first level feature maps
autocorr_output_pix=5
_feature_extractor = texture_statistics_gabor.texture_feature_extractor(_fmaps_fn_complex, _fmaps_fn_simple, \
                                        sample_batch_size=sample_batch_size, autocorr_output_pix=autocorr_output_pix, \
                                        n_prf_sd_out=n_prf_sd_out, aperture=aperture, \
                                        feature_types_exclude=feature_types_exclude, do_varpart=do_varpart, \
                                        group_all_hl_feats=group_all_hl_feats, device=device)      
feature_info = [_feature_extractor.feature_column_labels, _feature_extractor.feature_types_include]


In [50]:
feature_info

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 ['complex_feature_means']]

In [30]:

# Set up the pyramid
_fmaps_fn = texture_statistics_pyramid.steerable_pyramid_extractor(pyr_height = n_sf, n_ori = n_ori)
# Initialize the "texture" model which builds on first level feature maps
_feature_extractor2 = texture_statistics_pyramid.texture_feature_extractor(_fmaps_fn,sample_batch_size=sample_batch_size, feature_types_exclude=feature_types_exclude, n_prf_sd_out=n_prf_sd_out, aperture=aperture, do_varpart = do_varpart, \
                          group_all_hl_feats = group_all_hl_feats, device=device)

_feature_extractor = combined_feature_extractor([_feature_extractor, _feature_extractor2], \
                                                                       ['texture_gabor','texture_pyramid'], do_varpart = do_varpart)


[]


In [9]:
# # add an intercept
# add_bias=True
# # determines whether to shuffle before separating the nested heldout data for lambda and param selection. 
# # always using true.
# shuffle=True 
# best_losses, best_lambdas, best_params = fwrf_fit.fit_fwrf_model(trn_stim_data, trn_voxel_data, _feature_extractor, models, \
#                                                lambdas, zscore=zscore_features, add_bias=add_bias, \
#                                                voxel_batch_size=voxel_batch_size, holdout_size=holdout_size, \
#                                                shuffle=shuffle, shuff_rnd_seed=shuff_rnd_seed, device=device, \
#                                                                                debug=debug)


In [9]:
_feature_extractor.feature_types_include

['complex_feature_means']

In [19]:
_feature_extractor.init_for_fitting((224,224))
partial_masks, partial_version_names = _feature_extractor.get_partial_versions()
         

In [14]:
_feature_extractor.modules

[combined_feature_extractor(),
 texture_feature_extractor(
   (fmaps_fn): steerable_pyramid_extractor()
 )]

In [12]:
len(_feature_extractor.feature_group_names)

1

In [54]:
_feature_extractor.feature_types_include

['complex_feature_means']

In [56]:
_feature_extractor.feature_types_exclude = []

In [57]:
len(_feature_extractor.feature_types_exclude )

0

In [20]:
partial_version_names

['full_model',
 'just_pixel_stats',
 'just_complex_feature_means',
 'just_simple_feature_means',
 'just_complex_feature_autocorrs',
 'just_simple_feature_autocorrs',
 'just_complex_within_scale_crosscorrs',
 'just_simple_within_scale_crosscorrs',
 'just_complex_across_scale_crosscorrs',
 'just_simple_across_scale_crosscorrs',
 'leave_out_pixel_stats',
 'leave_out_complex_feature_means',
 'leave_out_simple_feature_means',
 'leave_out_complex_feature_autocorrs',
 'leave_out_simple_feature_autocorrs',
 'leave_out_complex_within_scale_crosscorrs',
 'leave_out_simple_within_scale_crosscorrs',
 'leave_out_complex_across_scale_crosscorrs',
 'leave_out_simple_across_scale_crosscorrs']

In [14]:
val_cc, val_r2 = fwrf_predict.validate_fwrf_model(best_params, models, val_voxel_data, val_stim_data, _feature_extractor, \
                                   sample_batch_size=sample_batch_size, voxel_batch_size=voxel_batch_size, debug=debug, dtype=fpX)
           

Clear maps fn
Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Computing complex cell features...
time elapsed = 3.73150
SKIPPING HIGHER-ORDER CORRELATIONS...
time elapsed = 0.00016
Final size of features concatenated is [62 x 16]
Feature types included are:
['complex_feature_means']
Getting features for prf 1: [x,y,sigma] is [-0.49 -0.55 0.0400]
Computing complex cell features...
time elapsed = 3.48273
SKIPPING HIGHER-ORDER CORRELATIONS...
time elapsed = 0.00010
Final size of features concatenated is [62 x 16]
Feature types included are:
['complex_feature_means']
Clear maps fn
Getting predictions for voxels [0-99] of 66545

Evaluating version 0 of 2: full_model
Includes 16 features
number of zeros:
0
size of weights is:
torch.Size([100, 16])

Evaluating version 1 of 2: just_complex_feature_means
Includes 16 features
number of zeros:
0
size of weights is:
torch.Size([100, 16])
Getting predictions for voxels [100-199] of 66545

Evaluating version 0 of 2: full_model
Includ

In [19]:
val_r2[:,1]

array([-0.04290649, -0.04575768, -0.06381177, ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [11]:
'pixel' in _feature_extractor.feature_types_include

False